In [ ]:
%%capture
!pip install pyautogen==0.2.0b5

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "secret_key"

In [ ]:
import autogen
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
import openai
import yfinance as yf

In [ ]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]
    return price

In [ ]:
tools_list = [{
    "type": "function",
    "function": {
        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
}]

In [ ]:
config_list = [
    {
        "model" : "gpt-4-1106-preview",
        "api_key" : "secret_key"
    },
    {
        "model" : "gpt-4-vision-preview",
        "api_key" : "secret_key"
    }
]

In [ ]:
llm_config = {
    "tools": tools_list,
    "config_list" : config_list
}

In [ ]:
gpt_assistant = GPTAssistantAgent(
    name="Stock Assistant",
    instructions="""
    You are a Stock Expert.
    Reply TERMINATE when the task is solved and there is no problem
    """,
    llm_config=llm_config
)

In [ ]:
gpt_assistant.register_function(
    function_map={
        "get_stock_price": get_stock_price,
    }
)

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="MervinPraison",
    code_execution_config={
        "work_dir" : "coding",
    }
)

In [ ]:
user_proxy.initiate_chat(
    gpt_assistant,
    message="""
    What is the stock price of Apple?
    """
)

MervinPraison (to Stock Assistant):


    What is the stock price of Apple?
    

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_stock_price...


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Intermediate executing(get_stock_price, Sucess: True) : 189.7100067138672


Stock Assistant (to MervinPraison):

The current stock price of Apple (AAPL) is $189.71. 
TERMINATE


--------------------------------------------------------------------------------
Provide feedback to Stock Assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit
